# Context-free grammars and CKY parsing

In this assignment, you get to experiment with large-scale context-free grammar (CFG) parsing using Python and NLTK. More specifically, we are asking you to implement the Cocke-Kasami-Younger (CKY) algorithm for bottom-up CFG parsing, and apply it to the word and the parsing problem of English.

The ingredients are: the grammar, the test sentences, and the parser. We
provide the first two ingredients. Please implement the parser from scratch,
using NLTK only to represent grammars and trees.

## Data Loading

The grammar stems from a project dealing with implementing spoken language processing systems in the airline industry – the Airline Travel Information System (ATIS). The ATIS CFG is available in the NLTK data package, together with 98 test sentences. You can initialize the resources this way:

```
grammar = nltk.data.load("grammars/large_grammars/atis.cfg") # load the grammar
s = nltk.data.load("grammars/large_grammars/atis_sentences.txt") # load raw sentences
t = nltk.parse.util.extract_test_sentences(s) # extract test sentences
```

In [1]:
import os
import itertools
from collections import defaultdict

from typing import List, Set, Union, Optional, Tuple, Dict
from tqdm import tqdm

import nltk
from nltk import Tree
from nltk.draw.tree import TreeView
from nltk.grammar import CFG, Nonterminal

In [2]:
nltk.download('large_grammars')

# grammar = nltk.data.load("grammars/large_grammars/atis.cfg") # load the grammar
s = nltk.data.load("grammars/large_grammars/atis_sentences.txt") # load raw sentences
t = nltk.parse.util.extract_test_sentences(s) # extract test sentences

[nltk_data] Downloading package large_grammars to /root/nltk_data...
[nltk_data]   Unzipping grammars/large_grammars.zip.


At this point, t is a list of 2-tuples, one per sentence. The first element of each tuple is the tokenized sentence, and the second element is the number
of parses for that sentence according to the grammar. Note that this number
can be zero.

In [3]:
# explore loaded things
print(*[line + '\n' for line in s.split('\n')][:15])

##########################################################################################
 # The test set for the ATIS grammar is a randomly selected subset of 
 # the DARPA ATIS3 development test set. 
 # (The full ATIS corpus is distributed by the Linguistic Data Consortium). 
 #
 # The original file can be found at URL:
 # http://www.informatics.sussex.ac.uk/research/groups/nlp/carroll/cfg-resources/
 #
 # Modified for NLTK by Peter Ljunglöf <peter.ljunglof@heatherleaf.se>
 # Each line begins with the number of parse trees according to the grammar.
 ##########################################################################################
 
 2085 : i need a flight from charlotte to las vegas that makes a stop in saint louis .
 1380 : what is the cheapest one way flight from phoenix to san diego that arrives in the morning on thursday june second .
 50 : what is the cheapest one way flight from columbus to indianapolis .



In [4]:
print('test corpus size: ', len(t))
print('sample test item: ', t[0])

test corpus size:  98
sample test item:  (['i', 'need', 'a', 'flight', 'from', 'charlotte', 'to', 'las', 'vegas', 'that', 'makes', 'a', 'stop', 'in', 'saint', 'louis', '.'], 2085)


NLTK already implements a number of parsing algorithms (see the documentation of nltk.parse for the list). You can try one to see if you loaded the grammar correctly:

```
# initialize the parser
parser = nltk.parse.BottomUpChartParser(grammar) 

# parse all test sentences
for sentence in t: 
    parser.chart_parse(sentence[0])
```

In [5]:
# # initialize the parser
# base_parser = nltk.parse.BottomUpChartParser(grammar) 

# # parse some sentences
# for sentence in t[:1]: 
#     base_parser.chart_parse(sentence[0])

However, the NLTK version of the ATIS grammar is not in Chomsky normal form (CNF), which you will need for your CKY parser. Feel free to implement a conversion module for extra credit, but for your convenience, we have already converted the ATIS CFG into CNF; you can download it from http://www.coli.uni-saarland.de/~koller/materials/anlp/atis.zip. You
can then read the grammar from the file using CFG.fromstring() and utilize
the features of the nltk.grammar module on the resulting object.

In [6]:
cnf_grammar_filename = 'atis-grammar-cnf.cfg'

In [7]:
%%capture
if not os.path.isfile(cnf_grammar_filename):
    ! wget http://www.coli.uni-saarland.de/~koller/materials/anlp/atis.zip
    ! unzip atis.zip
    ! rm atis.zip

In [8]:
with open(cnf_grammar_filename, 'r') as input_f:
    lines = input_f.readlines()
    cnf_grammar = CFG.fromstring(lines)

In [9]:
# # initialize the parser
# parser = nltk.parse.BottomUpChartParser(cnf_grammar)

# # parse some sentences
# for sentence in t[:1]:
#     parser.chart_parse(sentence[0])

## Recognizer

Implement the CKY algorithm and use it as a recognizer.

That is, given an input sentence, the procedure should decide whether the
sentence is in the language of the CFG or not. Test the recognizer on the
ATIS test sentences (not all of which are grammatical), but also by feeding it
other sentences to see whether it properly rejects ungrammatical sentences
as well. Submit some of the ungrammatical sentences you tried.

```
Data structure: Ch(i,k) eventually contains {A | A ⇒* wi ... wk-1} (initially all empty).

for each i from 1 to n:
  for each production rule A → wi:
    add A to Ch(i, i+1)

for each width b from 2 to n:
  for each start position i from 1 to n-b+1:
    for each left width k from 1 to b-1:
      for each B ∈ Ch(i, i+k) and C ∈ Ch(i+k,i+b):
        for each production rule A → B C: 
          add A to Ch(i,i+b)
          
claim that w ∈ L(G) iff S ∈ Ch(1,n+1)
```

In [10]:
ChartBackpointer = Tuple[Union[Nonterminal, str], Optional[Nonterminal], int]
ChartElement = Dict[Nonterminal, List[ChartBackpointer]]

In [11]:
class Chart:
    def __init__(self, n: int):
        # we need only top-left half of the n*n matrix for the chart
        # items of the chart are key-value pairs where
        # key is a unique non-terminal and values are back pointer tuples (B, C, j)
        self._chart = [[defaultdict(list) for j in range(n - i)] for i in range(n)]
        self.n = n

    def get(self, i: int, k: int) -> ChartElement:
        """
        This function is a trick to map indices from pseudocode above
        to the underlying Python representation of the chart.

        """
        return self._chart[self.n - k + 1][i - 1]

    def update(self, i: int, k: int, new_element: Nonterminal, backpointer: ChartBackpointer):
        """
        Adds non-terminal element to chart cell with an optional backpointer.
        """
        return self._chart[self.n - k + 1][i - 1][new_element].append(backpointer)

    def create_parse_subtrees(self, non_terminal: Nonterminal, i: int, k: int,
                              memo: Dict[Tuple[Nonterminal, int, int], List[Tree]] = None) -> List[Tree]:
        if memo is None:
            memo = dict()
        elif (non_terminal, i, k) in memo:
            return memo[(non_terminal, i, k)]

        backpointers = self.get(i, k)[non_terminal]

        trees = []
        for left, right, j in backpointers:
            if right is None:
                trees.append(Tree(str(non_terminal), [left]))
            else:
                left_subtrees = self.create_parse_subtrees(left, i, j)
                right_subtrees = self.create_parse_subtrees(right, j, k)
                for left_child, right_child in itertools.product(left_subtrees, right_subtrees):
                    trees.append(Tree(str(non_terminal), [left_child, right_child]))

        memo[(non_terminal, i, k)] = trees
        return trees

    def create_parse_trees(self, non_terminal: Nonterminal) -> List[Tree]:
        return self.create_parse_subtrees(non_terminal, 1, self.n + 1, memo=dict())

In [12]:
class CKYParserRecogniser:
    def __init__(self, grammar: CFG):
        self.grammar = grammar

    def recognise(self, sentence: Union[str, List[str]]) -> bool:
        parse_chart = self.parse(sentence)
        # claim that w ∈ L(G) iff S ∈ Ch(1,n+1)
        parsing_result = self.grammar.start() in parse_chart.get(1, parse_chart.n + 1)
        return parsing_result

    def parse(self, sentence: Union[str, List[str]]) -> Chart:
        """
        Recognises if the sentence words are valid in terms
        of the chosen context-free grammar (CFG) in Chomsky normal form (CNF).

        Basically, this function translates pseudocode from the lecture,
        listed above, to the actual Python code.
        """

        if isinstance(sentence, str):
            sentence = [word.lower() for word in sentence.split(' ')]

        n = len(sentence)
        chart = Chart(n)

        # for each i from 1 to n:
        for i in range(1, n + 1):
            cur_word = sentence[i - 1].lower()
            # cnf_grammar.productions(rhs=sentence_words[i]) returns all
            # only productions with the given first item in the right-hand side.

            # for each production rule A → wi:
            for production_rule in self.grammar.productions(rhs=cur_word):
                # CNF guarantees that right-hand side is
                # either exactly one terminal or exactly two non-terminals.
                if len(production_rule.rhs()) > 1:
                    message = f'Provided grammar is not in CNF: {production_rule}'
                    raise Exception(message)

                # Therefore, we do not have to filter out production rules
                # given that 'cur_word' is a terminal.

                # add A to Ch(i, i+1)
                chart.update(i, i + 1, production_rule.lhs(), (cur_word, None, -1))

        # for each width b from 2 to n:
        for b in range(2, n + 1):
            # for each start position i from 1 to n-b+1:
            for i in range(1, n - b + 2):
                # for each left width k from 1 to b-1:
                for k in range(1, b):
                    # for each B ∈ Ch(i, i+k) and C ∈ Ch(i+k,i+b):
                    left_nonterminals = chart.get(i, i + k)
                    right_nonterminals = chart.get(i + k, i + b)
                    if right_nonterminals:
                        for left_nonterminal in left_nonterminals:
                            # for each production rule A → B C:
                            for candidate_rule in self.grammar.productions(rhs=left_nonterminal):
                                if candidate_rule.rhs()[1] in right_nonterminals:
                                    # add A to Ch(i,i+b)
                                    right_nonterminal = candidate_rule.rhs()[1]
                                    backpointer = (left_nonterminal, right_nonterminal, i + k)
                                    chart.update(i, i + b, candidate_rule.lhs(), backpointer)

        return chart

    def recognise_test_sentences(self, sentences: List[Union[List[str], str]]):
        for sent in sentences:
            parsing_result = self.recognise(sent)
            input_for_printing = ' '.join(sent) if not isinstance(sent, str) else sent

            print('Input sentence: ', input_for_printing)
            print('Parsing result: ', parsing_result)
            print()

    def extract_parse_trees(self, sentence: Union[List[str], str]) -> List[Tree]:
        """Parses passed sentence and creates parse trees based on the created chart."""
        parse_trees = self.parse(sentence).create_parse_trees(self.grammar.start())
        return parse_trees

In [13]:
cky_parser = CKYParserRecogniser(cnf_grammar)

sentence_recognise_results = [cky_parser.recognise(sentence) for sentence, num_parses in tqdm(t)]

100%|██████████| 98/98 [00:14<00:00,  6.77it/s]


In [14]:
print('Recognised count: ', sentence_recognise_results.count(True))
print('Non-recognised count: ', sentence_recognise_results.count(False))
print()

recognised_indices = [i for i in range(len(t)) if sentence_recognise_results[i]]
non_recognised_indices = [i for i in range(len(t)) if not sentence_recognise_results[i]]

for i in range(5):
    print('Recognised sentence: ', " ".join(t[recognised_indices[i]][0]))
    print('Non-recognised sentence: ', " ".join(t[non_recognised_indices[i]][0]))
    print()

Recognised count:  70
Non-recognised count:  28

Recognised sentence:  i need a flight from charlotte to las vegas that makes a stop in saint louis .
Non-recognised sentence:  what aircraft is this .

Recognised sentence:  what is the cheapest one way flight from phoenix to san diego that arrives in the morning on thursday june second .
Non-recognised sentence:  what flights are available between chicago and indianapolis next wednesday between eleven a.m. and one p.m .

Recognised sentence:  what is the cheapest one way flight from columbus to indianapolis .
Non-recognised sentence:  please book a one way coach fare from chicago to indianapolis on united flight two ninety two next wednesday .

Recognised sentence:  is there a flight from memphis to los angeles .
Non-recognised sentence:  show american flights after twelve p.m. from miami to chicago .

Recognised sentence:  please show me the flights from chicago to detroit that arrive at six p.m. next tuesday .
Non-recognised sentence:

In [15]:
my_test_sentences = [
    ['fly', 'can', 'i', 'to', 'miami', '.'],
    ['can', 'i', 'fly', 'to', 'miami', '.'],
    ['can', 'i', 'fly', 'to', 'las', 'vegas', '.'],
    ['show', 'me', 'the', 'plane', 'to', 'miami', '.'],
]

cky_parser.recognise_test_sentences(my_test_sentences)

Input sentence:  fly can i to miami .
Parsing result:  False

Input sentence:  can i fly to miami .
Parsing result:  False

Input sentence:  can i fly to las vegas .
Parsing result:  False

Input sentence:  show me the plane to miami .
Parsing result:  True



I do not know why but perfectly correct sentences like 'can i fly to miami .' are getting rejected. 

Whereas their rephrasings ('show me the plane to miami .') are getting accepted. 

I reckon this is the grammar-dependent issue.

In [16]:
my_test_sentences = [
    'what is the price .',
    'what is the price ?',
    'what is a price .',
    'is what a price .',
    'is price a what .',
    'is price a what .',
]

cky_parser.recognise_test_sentences(my_test_sentences)

Input sentence:  what is the price .
Parsing result:  True

Input sentence:  what is the price ?
Parsing result:  False

Input sentence:  what is a price .
Parsing result:  True

Input sentence:  is what a price .
Parsing result:  True

Input sentence:  is price a what .
Parsing result:  True

Input sentence:  is price a what .
Parsing result:  True



Two general comments based on results:

1. Short sentencies are easier to accept as parsed, since we have not so much production rules applied.
2. This grammar does not know question mark.

In [17]:
my_test_sentences = [
    'show me cheap flights to miami .',
    'show me flights cheap to miami .',
     
    'me flights cheap to miami show .',
]

cky_parser.recognise_test_sentences(my_test_sentences)

Input sentence:  show me cheap flights to miami .
Parsing result:  True

Input sentence:  show me flights cheap to miami .
Parsing result:  True

Input sentence:  me flights cheap to miami show .
Parsing result:  False



Absolutely correct sentence is parsed ('show me cheap flights to miami .').

Additionally, the slightly ungrammatical sentence ('show me flights cheap to miami .') is parsed either. I assume that the grammar here knows transition like NP -> N ADJ as well as NP -> ADJ N. The other parts of the parsing algorithm will be the same.

Nevertheless, if we make the sentence 'too' ungrammatical ('me flights cheap to miami show .'), it is no more getting parsed.

## Parser

Now extend your CKY recognizer into a parser by adding backpointers. Also implement a function that extracts the set of all parse trees from the backpointers in the chart. Feel free to use the NLTK module
nltk.tree for this purpose; notice that only ImmutableTrees can be used
as elements of Python sets, whereas raw Trees cannot.

In [18]:
for sentence in my_test_sentences:
    parse_trees = cky_parser.extract_parse_trees(sentence)
    print(sentence, len(parse_trees))
    if parse_trees:
        parse_trees[0].pretty_print()

show me cheap flights to miami . 1
        SIGMA                                                            
    ______|_____________________                                          
   |                           EEA                                       
   |       _____________________|________                                 
   |      |                             EEB                              
   |      |              ________________|______________                  
   |      |             |                              EEC               
   |      |             |                        _______|__________       
   |      |           NP_NNS                  PP_NPS               |     
   |      |       ______|_______          ______|_______           |      
VERB_VB NP_PPO AJP_JJ        NOUN_NNS PREP_IN        NOUN_NPS pt_char_per
   |      |      |              |        |              |          |      
  show    me   cheap         flights     to           miami        .   

In [19]:
parse_trees_for_test_dataset = []

with open('parsing_results.txt', 'w') as output_f:
    for sentence_words, ground_truth_num_of_parse_trees in tqdm(t):
        parse_trees = cky_parser.extract_parse_trees(sentence_words)
        parse_trees_for_test_dataset.append(parse_trees)
        # check your parse tree counts against the counts in NLTK
        assert len(parse_trees) == ground_truth_num_of_parse_trees
        output_f.write(f"{' '.join(sentence_words)}\t{len(parse_trees)}\n")

100%|██████████| 98/98 [00:17<00:00,  5.50it/s]


In [20]:
small_parse_trees = [(i, parse_trees) for i, parse_trees in enumerate(parse_trees_for_test_dataset)
                     if 2 <= len(parse_trees) <= 5]
print(len(small_parse_trees))

14


In [23]:
tree_views_available = True

try:
    small_parse_trees[0][1][0].draw()
except:
    # I am getting the following error in Colab:
    # TclError: no display name and no $DISPLAY environment variable
    # We still can use tree views printed to console
    tree_views_available = False

In [24]:
for test_sentence_index, trees in small_parse_trees:
    parsed_sentence = " ".join(t[test_sentence_index][0])
    print(f'{test_sentence_index}. {parsed_sentence} {len(trees)}')
    for i, tree in enumerate(trees):
        if tree_views_available:
            tree.draw()

            # # there is an opportunity to export tree views to the file 
            # # after several hacks :)
            # # https://stackoverflow.com/questions/23429117/saving-nltk-drawn-parse-tree-to-image-file
            # # https://stackoverflow.com/questions/28627473/error-for-convert-command-in-command-line
            
            # tree_filename = f"{test_sentence_index}.{i}"
            # TreeView(t)._cframe.print_to_file(f'{tree_filename}.ps')
            # os.system(f'convert {tree_filename}.ps {tree_filename}.png')
        else:
            tree.pretty_print()

    print('***********************************************')

15. can you tell me about the flights from saint petersburg to toronto again . 3
         SIGMA                                                                                                                  
    _______|_______                                                                                                              
   |              BJG                                                                                                           
   |        _______|______                                                                                                       
   |       |             BJH                                                                                                    
   |       |        ______|____________________                                                                                  
   |       |       |                          BJI                                                                               
   |       | 

## Submission guideline

Submit your code, outputs, and a README file. The outputs should consist
of at least:

(1) your parsing results (number of parses per sentence), as a
text file with one line of the form sentence\t#parses for each test sentence,
where \t is a tab character; 

(2) pictures of the parse trees for an ATIS test
sentence with two to five parses. You can visualize an NLTK tree using its
draw method, and check your parse tree counts against the counts in NLTK.

## Extra credit 

If you still have time left, here’s a project for extra credit.

Perhaps it has occurred to you that it is quite wasteful to compute all parse
trees just to find out how many parse trees there are. Figure out how to
compute the number of parse trees for an entry A ∈ Ch(i, k) from your chart
with backpointers, without actually computing these parse trees. Verify that
you get the correct results, and compare the efficiency of your new procedure
to your earlier solution.